In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR='/home/csunix/ml17z2x/Downloads/practice/cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['readme.html', 'data_batch_2', 'batches.meta', 'data_batch_1', 'data_batch_3', 'data_batch_4', 'test_batch', 'data_batch_5']


In [11]:
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))


[[22. 28.]
 [49. 64.]]


In [12]:
tf.test.is_gpu_available()

False

In [ ]:
def load_data(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f,encoding='bytes')
        return data[b'data'], data[b'labels']


In [ ]:
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        #print(all_data)            
        self._data = np.vstack(all_data)
        #print(self._data)
        
        self._data = self._data / 255
        #print(self._data)
        self._labels = np.hstack(all_labels)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        
        #print(self._data.shape)
        #print(self._labels.shape)
        if self._need_shuffle:
            self._shuffle_data()
    
    
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        #print(self._indicator)
        #print(batch_size)
        if end_indicator > self._num_examples:
            #print(end_indicator)
            #print(self._num_examples)
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
                
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
            
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range (1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

#print(type(train_filenames))
#print(train_filenames)
train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

#batch_data, batch_labels = train_data.next_batch(100)
#print(batch_data)
#print(batch_labels)

In [ ]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, 3072]) #placeholder 占位图  ,None 表示样本数不确定
y = tf.placeholder(tf.int64,[None])



#w (3072,10)
w = tf.get_variable('w', [x.get_shape()[-1], 10], initializer = tf.random_normal_initializer(0,1))
#b (1, )
b = tf.get_variable('b', [10], initializer = tf.constant_initializer(0.0))


#y_ (None,3072) * (3072,10) = (None,10)
y_ = tf.matmul(x,w) + b
 
p_y = tf.nn.softmax(y_)
y_one_hot = tf.one_hot(y, 10, dtype=tf.float32)


loss = tf.reduce_mean(tf.square(y_one_hot - p_y))

"""
二分类用sigmoid 多分类用softmax
#p_y_1 (None,1)
p_y_1 = tf.nn.sigmoid(y_) 

#y_reshaped (None,1)
y_reshaped = tf.reshape(y, (-1, 1))
y_reshaped_float = tf.cast(y_reshaped, tf.float32)

loss = tf.reduce_mean(tf.square(y_reshaped_float - p_y_1))

#bool
predict = p_y_1 > 0.5
correct_prediction = tf.equal(tf.cast(predict, tf.int64), y_reshaped)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

"""

predict = tf.argmax(y_, 1)
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))



with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)



In [ ]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100


with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict={x: batch_data, y: batch_labels})
        
        if (i+1) % 500 == 0:
            print('[Train]Step: %d, loss: %4.5f, acc: %4.5f' % (i+1, loss_val, acc_val))
            
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict = {x: test_batch_data, y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test]Step: %d, acc: %4.5f' % (i+1,test_acc))
            